## 1. Gerekli Kütüphanelerin Yüklenmesi ve Veri Setinin Okunması

In [2]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix

# CSV okuma
df = pd.read_csv("C://Users//Yusuf//OneDrive//Masaüstü//offensive-language-detection//cümle_train.csv", sep='|')

# Gerekli sütun kontrolü
required_columns = ["text", "is_offensive", "target"]
assert all(col in df.columns for col in required_columns), "Eksik sütun var!"

# Temizleme
df = df[required_columns].dropna()
df = df[df["is_offensive"].isin([0, 1])]
df.head()

W0621 01:58:43.907000 14000 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


,text,is_offensive,target
0,çürük dişli,1,INSULT
1,Bu adamın islama ve müslümanlara verdiği zarar...,1,RACIST
2,erkekler zora gelmez,1,SEXIST
3,Utanmazın götüne kazık sokmuşlar bu tıkırtı ne...,1,PROFANITY
4,otomasyon< sistemlerine= doğrudan bağlanabilir,0,OTHER


## 2. Tokenizer ve Dataset Sınıfı Tanımlama

In [3]:
MODEL_NAME = "dbmdz/bert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = list(texts)
        self.labels = list(labels)
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        encoding = self.tokenizer(
            str(self.texts[idx]),
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item["labels"] = torch.tensor(int(self.labels[idx]))
        return item

c:\Users\Yusuf\Anaconda3\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## 3. Eğitim ve Test Setlerinin Oluşturulması (Binary Sınıflandırma için)

In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["is_offensive"])
train_dataset = TextDataset(train_df["text"], train_df["is_offensive"], tokenizer)
test_dataset = TextDataset(test_df["text"], test_df["is_offensive"], tokenizer)

## 4. Binary Model Eğitimi

In [5]:
model_binary = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

training_args = TrainingArguments(
    output_dir="./results-binary",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="no",
    logging_dir="./logs-binary",
    logging_steps=50,
    report_to=[]
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}

trainer = Trainer(
    model=model_binary,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()

c:\Users\Yusuf\Anaconda3\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Yusuf\Anaconda3\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_ba

  0%|          | 0/2524 [00:00<?, ?it/s]

c:\Users\Yusuf\Anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.4629, 'grad_norm': 0.8576146364212036, 'learning_rate': 4.9009508716323296e-05, 'epoch': 0.04}
{'loss': 0.2313, 'grad_norm': 0.10646834969520569, 'learning_rate': 4.8019017432646596e-05, 'epoch': 0.08}
{'loss': 0.2074, 'grad_norm': 24.114282608032227, 'learning_rate': 4.702852614896989e-05, 'epoch': 0.12}
{'loss': 0.3601, 'grad_norm': 4.719348907470703, 'learning_rate': 4.603803486529319e-05, 'epoch': 0.16}
{'loss': 0.305, 'grad_norm': 6.136990070343018, 'learning_rate': 4.5047543581616484e-05, 'epoch': 0.2}
{'loss': 0.1911, 'grad_norm': 4.337399005889893, 'learning_rate': 4.4057052297939784e-05, 'epoch': 0.24}
{'loss': 0.2006, 'grad_norm': 0.1265583485364914, 'learning_rate': 4.306656101426308e-05, 'epoch': 0.28}
{'loss': 0.1701, 'grad_norm': 0.13188447058200836, 'learning_rate': 4.207606973058637e-05, 'epoch': 0.32}
{'loss': 0.2072, 'grad_norm': 0.8634711503982544, 'learning_rate': 4.1085578446909665e-05, 'epoch': 0.36}
{'loss': 0.1994, 'grad_norm': 3.396576404571533, 'lea

  0%|          | 0/158 [00:00<?, ?it/s]

...

  0%|          | 0/158 [00:00<?, ?it/s]

{'eval_loss': 0.15380673110485077, 'eval_accuracy': 0.9675118858954042, 'eval_precision': 0.9684439608269858, 'eval_recall': 0.9866962305986696, 'eval_f1': 0.9774848984074684, 'eval_runtime': 128.3618, 'eval_samples_per_second': 19.663, 'eval_steps_per_second': 1.231, 'epoch': 2.0}
{'train_runtime': 5042.2601, 'train_samples_per_second': 4.003, 'train_steps_per_second': 0.501, 'train_loss': 0.12753593168338015, 'epoch': 2.0}


c:\Users\Yusuf\Anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/158 [00:00<?, ?it/s]

{'eval_loss': 0.15380673110485077,
 'eval_accuracy': 0.9675118858954042,
 'eval_precision': 0.9684439608269858,
 'eval_recall': 0.9866962305986696,
 'eval_f1': 0.9774848984074684,
 'eval_runtime': 130.0282,
 'eval_samples_per_second': 19.411,
 'eval_steps_per_second': 1.215,
 'epoch': 2.0}

## 5. Binary Sınıflandırma için Detaylı Rapor

In [6]:
preds = trainer.predict(test_dataset)
y_true = preds.label_ids
y_pred = np.argmax(preds.predictions, axis=1)
print(classification_report(y_true, y_pred))

c:\Users\Yusuf\Anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/158 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.97      0.92      0.94       720
           1       0.97      0.99      0.98      1804

    accuracy                           0.97      2524
   macro avg       0.97      0.95      0.96      2524
weighted avg       0.97      0.97      0.97      2524



## 6. Multi-Class Sınıflandırma (Saldırganlık Türleri)

In [7]:
off_df = df[df["is_offensive"] == 1].copy().reset_index(drop=True)
target2id = {label: i for i, label in enumerate(off_df["target"].unique())}
id2target = {i: label for label, i in target2id.items()}
off_df["target_id"] = off_df["target"].map(target2id)

train_off, test_off = train_test_split(off_df, test_size=0.2, random_state=42, stratify=off_df["target_id"])
train_dataset_off = TextDataset(train_off["text"], train_off["target_id"], tokenizer)
test_dataset_off = TextDataset(test_off["text"], test_off["target_id"], tokenizer)

model_multi = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(target2id))

training_args_multi = TrainingArguments(
    output_dir="./results-multi",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="no",
    logging_dir="./logs-multi",
    logging_steps=50,
    report_to=[]
)

def compute_metrics_multi(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}

trainer_multi = Trainer(
    model=model_multi,
    args=training_args_multi,
    train_dataset=train_dataset_off,
    eval_dataset=test_dataset_off,
    compute_metrics=compute_metrics_multi
)

trainer_multi.train()
print("Multi-class test sonucu:")
print(trainer_multi.evaluate())
preds_multi = trainer_multi.predict(test_dataset_off)
y_true_multi = preds_multi.label_ids
y_pred_multi = np.argmax(preds_multi.predictions, axis=1)
print(classification_report(y_true_multi, y_pred_multi, target_names=[id2target[i] for i in range(len(id2target))]))
print("Confusion Matrix:")
print(confusion_matrix(y_true_multi, y_pred_multi))

etiket_aciklamalari = {
    "INSULT": "INSULT (Hakaret)",
    "SEXIST": "SEXIST (Cinsiyetçi)",
    "RACIST": "RACIST (Irkçı)",
    "PROFANITY": "PROFANITY (Küfürlü)",
    "OTHER": "OTHER (Diğer)"
}

c:\Users\Yusuf\Anaconda3\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Yusuf\Anaconda3\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_ba

  0%|          | 0/1804 [00:00<?, ?it/s]

c:\Users\Yusuf\Anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.9269, 'grad_norm': 1.8219565153121948, 'learning_rate': 4.861419068736142e-05, 'epoch': 0.06}
{'loss': 0.4265, 'grad_norm': 17.04303550720215, 'learning_rate': 4.722838137472284e-05, 'epoch': 0.11}
{'loss': 0.3099, 'grad_norm': 0.36944687366485596, 'learning_rate': 4.584257206208426e-05, 'epoch': 0.17}
{'loss': 0.3331, 'grad_norm': 7.2172532081604, 'learning_rate': 4.445676274944568e-05, 'epoch': 0.22}
{'loss': 0.341, 'grad_norm': 17.76565933227539, 'learning_rate': 4.30709534368071e-05, 'epoch': 0.28}
{'loss': 0.3782, 'grad_norm': 6.366988658905029, 'learning_rate': 4.168514412416852e-05, 'epoch': 0.33}
{'loss': 0.3479, 'grad_norm': 29.49399185180664, 'learning_rate': 4.0299334811529936e-05, 'epoch': 0.39}
{'loss': 0.3677, 'grad_norm': 32.474124908447266, 'learning_rate': 3.8913525498891355e-05, 'epoch': 0.44}
{'loss': 0.306, 'grad_norm': 17.153549194335938, 'learning_rate': 3.7527716186252774e-05, 'epoch': 0.5}
{'loss': 0.2725, 'grad_norm': 9.060556411743164, 'learning_rat

  0%|          | 0/113 [00:00<?, ?it/s]

...

  0%|          | 0/113 [00:00<?, ?it/s]

{'eval_loss': 0.22276248037815094, 'eval_accuracy': 0.9512195121951219, 'eval_precision': 0.8830568781908221, 'eval_recall': 0.9058561458030056, 'eval_f1': 0.8929810848041434, 'eval_runtime': 90.6605, 'eval_samples_per_second': 19.898, 'eval_steps_per_second': 1.246, 'epoch': 2.0}


c:\Users\Yusuf\Anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/113 [00:00<?, ?it/s]

              precision    recall  f1-score   support

      INSULT       0.91      0.96      0.94       482
      RACIST       0.98      0.96      0.97       411
      SEXIST       0.96      0.96      0.96       420
   PROFANITY       0.97      0.93      0.95       477
       OTHER       0.59      0.71      0.65        14

    accuracy                           0.95      1804
   macro avg       0.88      0.91      0.89      1804
weighted avg       0.95      0.95      0.95      1804

Confusion Matrix:
[[463   4   7   7   1]
 [  8 396   2   2   3]
 [  9   3 405   2   1]
 [ 25   2   6 442   2]
 [  3   0   0   1  10]]


## 7. Multi-Class Sınıflandırma için Detaylı Rapor

In [8]:
preds_multi = trainer_multi.predict(test_dataset_off)
y_true_multi = preds_multi.label_ids
y_pred_multi = np.argmax(preds_multi.predictions, axis=1)
print(classification_report(y_true_multi, y_pred_multi, target_names=[id2target[i] for i in range(len(id2target))]))
print(confusion_matrix(y_true_multi, y_pred_multi))

c:\Users\Yusuf\Anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/113 [00:00<?, ?it/s]

              precision    recall  f1-score   support

      INSULT       0.91      0.96      0.94       482
      RACIST       0.98      0.96      0.97       411
      SEXIST       0.96      0.96      0.96       420
   PROFANITY       0.97      0.93      0.95       477
       OTHER       0.59      0.71      0.65        14

    accuracy                           0.95      1804
   macro avg       0.88      0.91      0.89      1804
weighted avg       0.95      0.95      0.95      1804

[[463   4   7   7   1]
 [  8 396   2   2   3]
 [  9   3 405   2   1]
 [ 25   2   6 442   2]
 [  3   0   0   1  10]]


## 8. Kullanıcıdan Cümle Alarak Tahmin

In [ ]:
while True:
    text = input("Bir cümle girin (çıkmak için 'q'): ")
    if text.lower() == 'q':
        break
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    with torch.no_grad():
        out = model_binary(**inputs)
        binary_pred = torch.argmax(out.logits, dim=1).item()
        if binary_pred == 0:
            print("Saldırgan DEĞİL.")
        else:
            print("Saldırgan! Türü tahmin ediliyor...")
            out2 = model_multi(**inputs)
            multi_pred = torch.argmax(out2.logits, dim=1).item()
            print(f"Tahmini saldırganlık türü: {id2target[multi_pred]}")